In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\box_square_functions.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 2000.0; tspan = [0.0, t]

# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]
U0 = u0_hom[index_hom]

u0 = [0.0, 0.0, 0.0]
ϵ_box = 1.0e-4;

In [3]:
number_points_on_side = 10
total_count_points = number_points_on_side * 2

length_p = 3
start_p = -1.700
end_p = -1.71

size_vector = length_p * total_count_points

60

In [19]:
prange =  range(1, 3, step = 1)

dots_u0 = zeros(total_count_points, 3)
check_events = Vector{Bool}(undef, total_count_points)
time_events = zeros(total_count_points)
dots_on_event = zeros(total_count_points, 3)
αs = zeros(total_count_points, 3)
norms = zeros(total_count_points, 3)
data = [dots_u0, check_events, time_events, dots_on_event, αs, norms];

In [20]:
p = SA[α, τ, τD, τy, J, xthr, ythr, u0_hom[3], ΔU0, β, I0_hom[3]]

fixedpoint = get_fixed_point(TM, jacob_TM_, p, u0)
A = get_matrix(fixedpoint[1], p, jacob_TM_, ϵ_box, 2)

dots_right_side, array_α_vec_right,
dots_left_side, array_α_vec_left,
dots_up_side, array_α_vec_up,
dots_down_side, array_α_vec_down = get_arrays_dots(number_points_on_side, 3)

fill_side_square(ϵ_box, fixedpoint[1], number_points_on_side, A, 
dots_right_side, array_α_vec_right, dots_left_side, array_α_vec_left,
dots_up_side, array_α_vec_up, dots_down_side, array_α_vec_down)


condition = make_event(fixedpoint[1], ϵ_box, A)
cb = ContinuousCallback(condition, nothing, affect!)

up_and_down_side = cat(dots_up_side, dots_down_side, dims = 1)

trajectory_from_side(p, A, up_and_down_side, total_count_points, fixedpoint[1], cb, data)

In [21]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 3; number of no event: 17


In [7]:
idx, idy = 2, 3
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1])

scatter!(αs[:, idx], αs[:, idy], color = :blue, markersize = 7.0)
scatter!(0.0, 0.0, color = :black, markersize = 7.0)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)